<a href="https://colab.research.google.com/github/StefaniaGutu/OffensiveLanguageClassification/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
import string
import spacy
import pickle
from os.path import exists

In [49]:
df = pd.read_csv('sample_data/labeled_data.csv')
print(df)

       Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                                   tweet  
0      !!! RT @m

In [50]:
dataset_labels = df["class"]
dataset_tweets = df["tweet"]

Preprocesare

In [63]:
def preprocess(dataset):
    # remove @user
    tweets_no_user = [re.sub(r'@[\w]*', '', tweet) for tweet in dataset_tweets]

    # remove URL
    tweets_no_links = [re.sub(r'http\S+', '', tweet) for tweet in tweets_no_user]

    # remove hashtags
    tweets_no_hashtags = [re.sub(r'#[a-zA-Z0-9_]+','', tweet) for tweet in tweets_no_links]

    # remove &_;
    tweets_no_apersand = [re.sub(r'&[a-zA-Z0-9_]+','', tweet) for tweet in tweets_no_hashtags]

    # remove Punctuations and Special Characters
    tweets_no_punct = [tweet.translate(str.maketrans('', '', string.punctuation)) for tweet in tweets_no_apersand]

    # remove Numbers
    tweets_no_digits = [re.sub(r'\d+', '', tweet) for tweet in tweets_no_punct]

    # Convert to lower case
    tweets_lower = [tweet.lower() for tweet in tweets_no_digits]

    # Convert more than 2 letter repetitions to 2 letter
    tweets_no_repetitions = [re.sub(r'(.)\1+', r'\1\1', tweet) for tweet in tweets_lower]

    # Replace multiple spaces with a single space
    tweets_no_spaces = [re.sub(r'\s+', ' ', tweet) for tweet in tweets_no_repetitions]

    # Lemma
    nlp = spacy.load('en_core_web_sm')
    all_words_lemma = [[word.lemma_ for word in nlp(sent)] for sent in tweets_no_spaces]

    # remove stopwords
    stop_words_spacy = nlp.Defaults.stop_words
    all_words_without_stops = [[word for word in sent if word not in stop_words_spacy] for sent in all_words_lemma]

    # remove Short Words
    words_no_shortWords = [[word for word in tweet if len(word)>2] for tweet in all_words_without_stops]
    tweets_no_shortWords = [word for word in words_no_shortWords]

    return tweets_no_shortWords

In [64]:
preprocessed_dataset = preprocess(dataset_tweets)

print(preprocessed_dataset[:5])

[['woman', 'complain', 'clean', 'house', 'man', 'trash'], ['boy', 'dat', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'place'], ['dawg', 'fuck', 'bitch', 'start', 'cry', 'confuse', 'shit'], ['look', 'like', 'tranny'], ['shit', 'hear', 'true', 'faker', 'bitch', 'tell']]


In [65]:
preprocessed_dataset[:15]

[['woman', 'complain', 'clean', 'house', 'man', 'trash'],
 ['boy', 'dat', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'place'],
 ['dawg', 'fuck', 'bitch', 'start', 'cry', 'confuse', 'shit'],
 ['look', 'like', 'tranny'],
 ['shit', 'hear', 'true', 'faker', 'bitch', 'tell'],
 ['shit', 'blow', 'meclaim', 'faithful', 'somebody', 'fuck', 'hoe'],
 ['sit', 'hate', 'bitch', 'shit'],
 ['cause', 'tired', 'big', 'bitch', 'come', 'skinny', 'girl'],
 ['bitch'],
 ['hobby', 'include', 'fight', 'mariam', 'bitch'],
 ['keek', 'bitch', 'curve', 'lol', 'walk', 'conversation', 'like', 'smh'],
 ['murda', 'gang', 'bitch', 'gang', 'land'],
 ['hoe', 'smoke', 'loser', 'yea'],
 ['bad', 'bitch', 'thing', 'like'],
 ['bitch']]

In [66]:
# Save preprocessed dataset
pck = open("./pickles/preprocessed_dataset.pck", "wb")
pickle.dump({
    'preprocessed_dataset': preprocessed_dataset,
    'dataset_labels': dataset_labels
}, pck)

In [70]:
preprocessed_dataset1 = []
dataset_labels1 = []

pck = open("./pickles/preprocessed_dataset.pck", "rb")
result = pickle.load(pck)
preprocessed_dataset1 = result['preprocessed_dataset']
dataset_labels1 = result['dataset_labels']

Split data in train, test, validation

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(preprocessed_dataset, dataset_labels, test_size=0.30, random_state=42)

X_test, X_validation, y_test, y_validation = train_test_split(X_val_test, y_val_test, test_size=0.50, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(X_validation))